In [14]:
import grpc 
import tensorflow as tf 
from tensorflow_serving.apis import predict_pb2 
from tensorflow_serving.apis import prediction_service_pb2_grpc 

In [15]:
host = 'localhost:8500' 
channel = grpc.insecure_channel(host) 
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [16]:
from keras_image_helper import create_preprocessor 
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [17]:
url = "http://bit.ly/mlbookcamp-pants" 
X = preprocessor.from_url(url)

In [18]:
def np_to_protobuf(data): 
    return tf.make_tensor_proto(data, shape=data.shape) 

In [19]:
pb_request = predict_pb2.PredictRequest() 
pb_request.model_spec.name = 'clothing-model' 
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))
pb_result = stub.Predict(pb_request, timeout=20.0) 
pred = pb_result.outputs['dense_7'].float_val 

In [20]:
labels = [ 
    'dress', 
    'hat', 
    'longsleeve', 
    'outwear', 
    'pants', 
    'shirt', 
    'shoes', 
    'shorts', 
    'skirt', 
    't-shirt' 
] 
result = {c: p for c, p in zip(labels, pred)}

In [21]:
print(result)

{'dress': -1.8682907819747925, 'hat': -4.761244773864746, 'longsleeve': -2.316983461380005, 'outwear': -1.062570333480835, 'pants': 9.887161254882812, 'shirt': -2.812433958053589, 'shoes': -3.666283130645752, 'shorts': 3.200361728668213, 'skirt': -2.6023383140563965, 't-shirt': -4.835046768188477}
